# Laboratorio 8 
    - Francis Aguilar - 22243
    - César López - 22535
    - Angela García -22869 


enlace al repositorio: https://github.com/angelargd8/lab8-ds


In [0]:

# Cargar el archivos
# /Volumes/workspace/default/data_lab8/data lab8/fallecidos/2013 vehiculos involucrados.xlsx
path_fallecidos = "/Volumes/workspace/default/data_lab8/data lab8/fallecidos/"
path_hechos = "/Volumes/workspace/default/data_lab8/data lab8/hechos y transito/"
path_vehiculos = "/Volumes/workspace/default/data_lab8/data lab8/vehiculos involucrados/"

In [0]:
# %pip install openpyxl

### 1. Mostrar cuántos registros hay en cada tabla (hechos, vehículos, fallecidos, lesionados). Muestre algunos resultados con la función .show(). Genere un describe y summary para aquellas columnas que considere importantes según cada archivo.

Primero convertimos en csv los xlsx para poder procesarlos con spark


In [0]:
import pandas as pd
import glob
import os

paths = [path_fallecidos, path_hechos, path_vehiculos]
# paths = [path_fallecidos]

# convertimos los datos a csv antes de leerlos con spark 
for path in paths: 
    for file in glob.glob(os.path.join(path, "*.xlsx")):
        df = pd.read_excel(file)
        csv_path = file.replace(".xlsx", ".csv")
        df.to_csv(csv_path, index=False)



In [0]:

# Listar los CSVs en la carpeta
files_fallecidos = [f.path for f in dbutils.fs.ls(path_fallecidos) if f.name.endswith(".csv")]
files_hechos = [f.path for f in dbutils.fs.ls(path_hechos) if f.name.endswith(".csv")]
files_vehiculos = [f.path for f in dbutils.fs.ls(path_vehiculos) if f.name.endswith(".csv")]


def mostrar_columnas(files):
    columnas_unicas = set()

    for file in files:
        try:
            df = spark.read.option("header", "true").csv(file, inferSchema=True)
            count = df.count()
            print(f"ARCHIVO: {file}")
            print(f"   Columnas ({len(df.columns)}): {df.columns}")
            print(f"   Registros: {count}")
            print("-" * 80)

            # Agregar columnas al conjunto (evita duplicados)
            columnas_unicas.update(df.columns)

        except Exception as e:
            print(f"Error al leer el archivo {file}: {e}")

    # Convertir a lista ordenada alfabéticamente
    columnas_ordenadas = sorted(columnas_unicas)
    
    print("\nLista de todas las columnas únicas (ordenadas alfabéticamente):")
    for col in columnas_ordenadas:
        print(f" - {col}")


In [0]:
# visualizamos las columnas antes de realizar cambios 
mostrar_columnas(files_fallecidos)

In [0]:
%pip install unidecode

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import lit
import unidecode


archivos = []

for file in files_fallecidos:
    df = spark.read.option("header", "true").csv(file, inferSchema=True)
    if file == "2014 fallecidos y lesionados.csv":
        df = df.drop("num_corre", "corre_base")
     # ---  Eliminar columnas específicas si aparecen ---
    columnas_a_eliminar = [
        "g_edad_2", "g_edad_60ymás", "g_edad_80ymás",
        "Otro_g_edad_fall_les", "edad_quinquenales",
        "g_hora_5", "g_modelo_veh", "int_o_noint",
        "área_geo_ocu", "g_hora"
    ]
    for col in columnas_a_eliminar:
        if col in df.columns:
            df = df.drop(col)

    # --- Renombrar columnas relacionadas con num_correlativo ---
    corre_cols = ["Núm_corre", "num_corre", "num_hecho", "núm_corre"]
    for col in corre_cols:
        if col in df.columns:
            df = df.withColumnRenamed(col, "num_correlativo")

    # --- Reemplazar 'fall_les' por 'Fallecidos_Lesionados' ---
    if "fall_les" in df.columns:
        df = df.withColumnRenamed("fall_les", "Fallecidos_Lesionados")

    # --- Quitar tildes de los nombres de columnas ---
    new_columns = [unidecode.unidecode(c) for c in df.columns]
    df = df.toDF(*new_columns)

    # --- Unificar edad_per y edad_pil en una sola columna ---
    # Si ambas existen, preferimos mantener 'edad_per'
    if "edad_pil" in df.columns and "edad_per" in df.columns:
        df = df.drop("edad_pil")
    elif "edad_pil" in df.columns:
        df = df.withColumnRenamed("edad_pil", "edad_per")

    if "sexo_pil" in df.columns:
        df = df.withColumnRenamed("sexo_pil", "sexo_per")

    # --- Agregar columna con el nombre del archivo ---
    nombre_archivo = file.split("/")[-1]  # extrae solo el nombre sin la ruta
    df = df.withColumn("archivo_origen", lit(nombre_archivo))


    archivos.append(df)
    
    nombre_salida = os.path.splitext(nombre_archivo)[0] + "_limpio.csv"
    ruta_salida = os.path.join(path_fallecidos+'data_clean/', nombre_salida)
    df.coalesce(1).write.mode("overwrite").option("header", "true").csv(ruta_salida)

archivos[0].show(5)
print("Columnas finales:", archivos[0].columns)


In [0]:
df_fallecidos = (
    spark.read
    .option("header", True)
    .option("delimiter", ",")
    .option("recursiveFileLookup", "true")  # 🔍 busca dentro de subcarpetas
    .csv(path_fallecidos + "data_clean/")
)
df_fallecidos.printSchema()   # Muestra las columnas y tipos de datos
total_filas = df_fallecidos.count()   # Cuenta las filas

print(f"El DataFrame tiene {total_filas} registros.")
print(len(df_fallecidos.columns))


In [0]:
#Revisión de valores únicos y duplicados
for c in df_fallecidos.columns:
    print(c, "→", df_fallecidos.select(c).distinct().count())

In [0]:
for c in df_fallecidos.columns:
    print(f"\n=== Columna: {c} ===")
    valores = df_fallecidos.select(c).distinct().collect()  # muestra hasta 20
    print([v[c] for v in valores])

In [0]:
mostrar_columnas(files_hechos)

In [0]:
mostrar_columnas(files_vehiculos)

In [0]:

df_hechos = spark.read.option("delimiter", ",").option("header", True).csv(path_hechos)
df_vehiculos = spark.read.option("delimiter", ",").option("header", True).csv(path_vehiculos)


In [0]:
df_fallecidos.show(5)

In [0]:
df_hechos.printSchema()   # Muestra las columnas y tipos de datos
total_filas = df_hechos.count()   # Cuenta las filas

print(f"El DataFrame tiene {total_filas} registros.")
print(len(df_hechos.columns))


In [0]:
df_hechos.show(5)

In [0]:
df_vehiculos.printSchema()   # Muestra las columnas y tipos de datos
total_filas = df_vehiculos.count()   # Cuenta las filas

print(f"El DataFrame tiene {total_filas} registros.")
print(len(df_vehiculos.columns))

In [0]:

df_vehiculos.show(5)

2. Identificar los años disponibles en cada tabla y validar si coinciden. 

3.  Mostrar los valores distintos de tipo de accidente.

4.  Calcular cuántos departamentos únicos aparecen en las bases. 

5. ¿Cuál es el total de accidentes por año y departamento? Apóyese de la función 
groupBy. Investigue la función display que tiene Databricks y muestre su resultado en 
formato de gráfico de barras. 

6. ¿Qué día de la semana registra más accidentes en 2024? Graficar con display en un
gráfico de columnas

En 2024 el dia que muestra mas accidentes es el dia domingo

7. Mostrar la distribución de accidentes por hora del día en el municipio de 
Guatemala. Graficar en un histograma. 